In [17]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('movie_metadata.csv')

to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']
          
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 

movie_data.country = movie_data.country.replace(to_replace='USA', value = 1)

for item in movie_data["country"]:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)

In [18]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

{'Sport', 'Short', 'Horror', 'War', 'Mystery', 'Sci-Fi', 'Reality-TV', 'Biography', 'Animation', 'Western', 'Adventure', 'Romance', 'News', 'History', 'Crime', 'Fantasy', 'Thriller', 'Musical', 'Game-Show', 'Drama', 'Film-Noir', 'Documentary', 'Comedy', 'Action', 'Family', 'Music'}


In [21]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']

for genre in genres_set:
    movie_data[genre] = pd.Series(genres_dict[genre])
    
print(movie_data)    

      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0                   855.0   
1                    40000.0                  5000.0                  1000.0   
2                    11000.0                   393.0                   161.0   
3                    27000.0                 23000.0                 23000.0   
4                      131.0                    12.0                     NaN   
5                      640.0                   632.0                   530.0   
6                    24000.0                 11000.0                  4000.0   
7                      799.0                   553.0                   284.0   
8                    26000.0                 21000.0                 19000.0   
9                    25000.0                 11000.0                 10000.0   
10                   15000.0                  4000.0                  2000.0   
11                   18000.0            

In [ ]:
movie_num = movie_num.fillna(value=0, axis=1)
ratings = movie_num['imdb_score'].values
del movie_num['imdb_score']
#print(movie_num.head())

#ratings_integers = [int(round(rating)) for rating in ratings]

X = movie_num.values
# Data Normalization
X_std = StandardScaler().fit_transform(X)

number_of_samples = len(ratings)
np.random.seed(0)
random_indices = np.random.permutation(number_of_samples)
num_training_samples = int(number_of_samples*0.75)
movie_num_training = X_std[random_indices[:num_training_samples]]
ratings_cont_training = ratings[random_indices[:num_training_samples]]


movie_num_test = X_std[random_indices[num_training_samples:]]
ratings_cont_test = ratings[random_indices[num_training_samples:]]
ratings_cont_training = list(ratings_cont_training)

# trainingdata:

# number_for_training = int(round(0.75*len(ratings)))

# movie_num_training = X_std[:number_for_training]
# #ratings_int_training = ratings_integers[:number_for_training]
# ratings_cont_training = ratings[:number_for_training]

# #testdata:

# movie_num_test = X_std[number_for_training:]
# #ratings_int_test = ratings_integers[number_for_training:]
# ratings_cont_test = ratings[number_for_training:]

In [ ]:
# k-nearest neighbor met integers voor ratings (is niet heel accuraat):

# neighbor = KNeighborsClassifier(n_neighbors = 1)
# neighbor.fit(movie_num_training, ratings_int_training)
# neighbor.score(movie_num_test, ratings_int_test)


# k-nearest neighbor voor continue ratings:

regressor = KNeighborsRegressor()
regressor.fit(movie_num_training, ratings_cont_training)
y_predict_train = regressor.predict(movie_num_training)
y_predict_train = list(y_predict_train)

error = 0

for i in range(len(ratings_cont_training)):
    error += (abs(y_predict_train[i]-ratings_cont_training[i]))/ratings_cont_training[i]
training_error = error/len(ratings_cont_training)*100

print("Train error = "+'{}'.format(training_error)+" percent"+" in knn algorithm")

y_predict_test = regressor.predict(movie_num_test)
y_predict_test = list(y_predict_test)

error = 0

for i in range(len(ratings_cont_test)):
    error += (abs(y_predict_test[i]-ratings_cont_test[i]))/ratings_cont_test[i]
test_error = error/len(ratings_cont_test)*100

print("Test error = "'{}'.format(test_error)+" percent"+" in knn algorithm")

In [ ]:
# Neural networks:

from sklearn.neural_network import MLPRegressor

neural = MLPRegressor(alpha = 0.00001)#activation='logistic')

neural.fit(movie_num_training, ratings_cont_training)
y_neural_1 = neural.predict(movie_num_training)
y_neural_2 = neural.predict(movie_num_test)
print(ratings_cont_test)
print(y_neural_2)
error = 0

for i in range(len(ratings_cont_training)):
    error += (abs(y_neural_1[i]-ratings_cont_training[i]))/ratings_cont_training[i]
training_error = error/len(ratings_cont_training)*100

print("Train error = "+'{}'.format(training_error)+" percent"+" in neural network algorithm")

error = 0

for i in range(len(ratings_cont_test)):
    error += (abs(y_neural_2[i]-ratings_cont_test[i]))/ratings_cont_test[i]
test_error = error/len(ratings_cont_test)*100

print("Test error = "'{}'.format(test_error)+" percent"+" in neural network algorithm")